In [1]:
import numpy  # needed (don't change it)
import importlib
import os
import socket
import sys
import setproctitle
import torch

import datetime
import uuid
from argparse import ArgumentParser


mammoth_path = '/home/jw7630/repos/CL-Task-Poisoning' #os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # nopep8
sys.path.append(mammoth_path)  # nopep8
sys.path.append(mammoth_path + '/datasets')  # nopep8
sys.path.append(mammoth_path + '/backbone')  # nopep8
sys.path.append(mammoth_path + '/models')  # nopep8


from datasets import NAMES as DATASET_NAMES  # nopep8
from datasets import ContinualDataset, get_dataset  # nopep8
from models import get_all_models, get_model  # nopep8

from utils.args import add_management_args
from utils.best_args import best_args
from utils.conf import set_random_seed
from utils.continual_training import train as ctrain
from utils.distributed import make_dp
from utils.training import train

def lecun_fix():
    # Yann moved his website to CloudFlare. You need this now
    from six.moves import urllib  # pyright: ignore
    opener = urllib.request.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/5.0')]
    urllib.request.install_opener(opener)

lecun_fix()

In [27]:
def main(args=None, log_filename=None):
    os.putenv("MKL_SERVICE_FORCE_INTEL", "1")
    os.putenv("NPY_MKL_FORCE_INTEL", "1")

    # Add uuid, timestamp and hostname for logging
    args.conf_jobnum = str(uuid.uuid4())
    args.conf_timestamp = str(datetime.datetime.now())
    args.conf_host = socket.gethostname()
    dataset = get_dataset(args)

    if args.batch_size is None:
        args.batch_size = dataset.get_batch_size()
    if hasattr(importlib.import_module('models.' + args.model), 'Buffer') and args.minibatch_size is None:
        args.minibatch_size = dataset.get_minibatch_size()

    backbone = dataset.get_backbone()
    loss = dataset.get_loss()
    model = get_model(args, backbone, loss, dataset.get_transform())


    # set job name
    setproctitle.setproctitle('{}_{}_{}'.format(args.model, args.buffer_size if 'buffer_size' in args else 0, args.dataset))

    if isinstance(dataset, ContinualDataset):
        train(model, dataset, args, log_filename)
    else:
        assert not hasattr(model, 'end_task') or model.NAME == 'joint_gcl'
        ctrain(args)

In [45]:
models = ["er", "der", "derpp", "xder", "er_ace"]
buffer_sizes = [200, 500, 2000, 5000]
label_flip_percentages = [0, 25, 50, 75]


In [61]:
import itertools

def argument_combinations(buffer_sizes, models, label_flip_percentages):
    """
    Generator that yields every combination of arguments.
    """
    for combination in itertools.product(buffer_sizes, models, label_flip_percentages):
        yield {
            "buffer_size": combination[0],
            "model": combination[1],
            "label_flip_percentage": combination[2],
        }

models = ["er", "der", "derpp", "xder", "er_ace"]
buffer_size = [200, 500, 2000, 5000]
label_flip_percentage = [0, 25, 50, 75]



# Gaussian noise fro input coruption, highest severity

In [57]:

for args in argument_combinations(buffer_sizes, models, label_flip_percentages):
    print(args)

TypeError: 'int' object is not iterable

In [60]:
models = ["er", "der", "derpp", "xder", "er_ace"]
buffer_sizes = [200, 500, 2000, 5000]
label_flip_percentages = [0, 25, 50, 75]

for args in argument_combinations(buffer_sizes, models, label_flip_percentages):
    print(args)

{'buffer_size': 200, 'image_poisoning_type': 'er', 'label_flip_percentage': 0}
{'buffer_size': 200, 'image_poisoning_type': 'er', 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 'er', 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 'er', 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 'der', 'label_flip_percentage': 0}
{'buffer_size': 200, 'image_poisoning_type': 'der', 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 'der', 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 'der', 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 'derpp', 'label_flip_percentage': 0}
{'buffer_size': 200, 'image_poisoning_type': 'derpp', 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 'derpp', 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 'derpp', 'label_flip_percentage': 75}
{'buffer_size': 200, 'image

In [62]:
for i in itertools.product(buffer_sizes, models, label_flip_percentages):
    print(i)

(200, 'er', 0)
(200, 'er', 25)
(200, 'er', 50)
(200, 'er', 75)
(200, 'der', 0)
(200, 'der', 25)
(200, 'der', 50)
(200, 'der', 75)
(200, 'derpp', 0)
(200, 'derpp', 25)
(200, 'derpp', 50)
(200, 'derpp', 75)
(200, 'xder', 0)
(200, 'xder', 25)
(200, 'xder', 50)
(200, 'xder', 75)
(200, 'er_ace', 0)
(200, 'er_ace', 25)
(200, 'er_ace', 50)
(200, 'er_ace', 75)
(500, 'er', 0)
(500, 'er', 25)
(500, 'er', 50)
(500, 'er', 75)
(500, 'der', 0)
(500, 'der', 25)
(500, 'der', 50)
(500, 'der', 75)
(500, 'derpp', 0)
(500, 'derpp', 25)
(500, 'derpp', 50)
(500, 'derpp', 75)
(500, 'xder', 0)
(500, 'xder', 25)
(500, 'xder', 50)
(500, 'xder', 75)
(500, 'er_ace', 0)
(500, 'er_ace', 25)
(500, 'er_ace', 50)
(500, 'er_ace', 75)
(2000, 'er', 0)
(2000, 'er', 25)
(2000, 'er', 50)
(2000, 'er', 75)
(2000, 'der', 0)
(2000, 'der', 25)
(2000, 'der', 50)
(2000, 'der', 75)
(2000, 'derpp', 0)
(2000, 'derpp', 25)
(2000, 'derpp', 50)
(2000, 'derpp', 75)
(2000, 'xder', 0)
(2000, 'xder', 25)
(2000, 'xder', 50)
(2000, 'xder', 75

In [64]:
import itertools
import argparse

models = ["er", "der", "derpp", "xder", "er_ace"]
buffer_sizes = [200]#[200, 500, 2000, 5000]
label_flip_percentages = [0]#[0, 25, 50, 75]

for buffer_size, model, label_flip_percentage in itertools.product(buffer_sizes, models, label_flip_percentages):
    args_dict = dict(
        dataset='seq-cifar100-label-poisoning',
        model=model,
        n_epochs=1,
        nowand=1,
        non_verbose=1,
        ignore_other_metrics=1,
        seed=42,
        n_label_flip_poisonings=1,
        poisoning_severity=5,
        classes_per_poisoning=0,
        buffer_mode="reservoir",
        n_image_poisonings=None, # fails without this
        debug_mode=0, # fails without this
        disable_log=0, # fails without this
        batch_size=128, # not in best args
        minibatch_size=128, # not in best args
        buffer_size=buffer_size,
        label_flip_percentage=label_flip_percentage
    )

    if buffer_size == 200:
        bs_config = 500
    elif buffer_size == 5000:
        bs_config = 2000
    else:
        bs_config = buffer_size

    optim_params = best_args['seq-cifar100'][model][bs_config]
    args_dict.update(optim_params)
    args = argparse.Namespace(**args_dict)
    file = f"results/label_poison/cifar_{model}_{buffer_size}_{label_flip_percentage}_(5-20).json"
    main(args, file)

Creating poisonings at tasks:  [10]
Namespace(dataset='seq-cifar100-label-poisoning', model='er', n_epochs=1, nowand=1, non_verbose=1, ignore_other_metrics=1, seed=42, n_label_flip_poisonings=1, poisoning_severity=5, classes_per_poisoning=0, buffer_mode='reservoir', n_image_poisonings=None, debug_mode=0, disable_log=0, batch_size=128, minibatch_size=128, buffer_size=200, label_flip_percentage=0, lr=0.1, optim_mom=0, optim_wd=0, conf_jobnum='6ce8db64-8d8f-4b3b-9cf9-21f0ffb49415', conf_timestamp='2024-12-29 14:34:37.022100', conf_host='rankine.cis.rit.edu')
New Classes: [0, 1, 2, 3, 4]
Poisoned Classes: []



/home/aa5264/Desktop/CL-Mammoth/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
[ 12-29 | 14:34 ] Task 1 | epoch 0

Metrics for 1 task(s): 	 [Class-IL Acc]: 50.6 % 	 [Task-IL Acc]: 50.6 %



New Classes: [5, 6, 7, 8, 9]
Poisoned Classes: []


[ 12-29 | 14:34 ] Task 2 | epoch 0

Metrics for 2 task(s): 	 [Class-IL Acc]: 26.7 % 	 [Task-IL Acc]: 44.6 %



New Classes: [10, 11, 12, 13, 14]
Poisoned Classes: []


...

[ 12-29 | 14:46 ] Task 18 | epoch 0

Metrics for 18 task(s): 	 [Class-IL Acc]: 10.8 % 	 [Task-IL Acc]: 54.73 %



New Classes: [90, 91, 92, 93, 94]
Poisoned Classes: []


[ 12-29 | 14:46 ] Task 19 | epoch 0

Metrics for 19 task(s): 	 [Class-IL Acc]: 8.44 % 	 [Task-IL Acc]: 52.65 %



New Classes: [95, 96, 97, 98, 99]
Poisoned Classes: []


[ 12-29 | 14:46 ] Task 20 | epoch 0

Metrics for 20 task(s): 	 [Class-IL Acc]: 8.34 % 	 [Task-IL Acc]: 53.58 %



In [7]:
import argparse
from tqdm import tqdm

for args in tqdm(argument_combinations(buffer_size, image_poisoning_type, label_flip_percentage)):
    args = argparse.Namespace(
        seed=None,
        notes=None,
        non_verbose=1,
        disable_log=0,
        validation=0,
        ignore_other_metrics=1,
        debug_mode=0,
        nowand=1,
        wandb_entity='regaz',
        wandb_project='mammoth',
        dataset='seq-cifar100-label-poisoning',
        model='der',
        lr=0.1,
        optim_wd=0.0,
        optim_mom=0.0,
        optim_nesterov=0,
        n_epochs=50,
        batch_size=32,
        distributed='no',
        n_image_poisonings=None,
        image_poisoning_type=None,
        n_label_flip_poisonings=None, 
        label_flip_percentage=None,
        poisoning_severity=1, 
        classes_per_poisoning=0, 
        buffer_size=5000, 
        minibatch_size=32, 
        buffer_mode='reservoir', #balanced
        #conf_jobnum='80cc9790-1cc7-4ef5-b5b8-7a2574ca06f7', 
        #conf_timestamp='2024-12-28 17:53:09.485930', 
        conf_host='rankine.cis.rit.edu',
        use_best_args=1,
        alpha=0.1
    )

    log_filename = (
        f"results/Task-Poisoning/{datetime.datetime.now().strftime('%m-%d-%y-%H-%M-%S')}-{args.dataset}-{args.model}-buf-{args.buffer_size}-severity-{args.poisoning_severity}-cpp-{args.classes_per_poisoning}"
        f"{'-poisoning-type-' + str(args.image_poisoning_type) if args.n_image_poisonings is not None else '-poisoning-percentage-' + str(args.label_flip_percentage) if args.n_label_flip_poisonings is not None else '-no-poisoning'}.json"
    )
    main(args, log_filename)

0it [00:00, ?it/s]


Namespace(seed=None, notes=None, non_verbose=1, disable_log=0, validation=0, ignore_other_metrics=1, debug_mode=0, nowand=1, wandb_entity='regaz', wandb_project='mammoth', dataset='seq-cifar100-label-poisoning', model='der', lr=0.1, optim_wd=0.0, optim_mom=0.0, optim_nesterov=0, n_epochs=50, batch_size=32, distributed='no', n_image_poisonings=None, image_poisoning_type=None, n_label_flip_poisonings=None, label_flip_percentage=None, poisoning_severity=1, classes_per_poisoning=0, buffer_size=5000, minibatch_size=32, buffer_mode='reservoir', conf_host='rankine.cis.rit.edu', use_best_args=1, alpha=0.1, conf_jobnum='3bceac65-f3c6-47e0-850f-011335a57607', conf_timestamp='2024-12-29 12:50:21.373578')


/home/aa5264/Desktop/CL-Mammoth/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
[ 12-29 | 12:50 ] Task 1 | epoch 0
[ 12-29 | 12:50 ] Task 1 | epoch 1
[ 12-29 | 12:50 ] Task 1 | epoch 2
[ 12-29 | 12:50 ] Task 1 | epoch 3
[ 12-29 | 12:50 ] Task 1 | epoch 4
[ 12-29 | 12:50 ] Task 1 | epoch 5
[ 12-29 | 12:50 ] Task 1 | epoch 6
[ 12-29 | 12:50 ] Task 1 | epoch 7
[ 12-29 | 12:50 ] Task 1 | epoch 8
[ 12-29 | 12:50 ] Task 1 | epoch 9
[ 12-29 | 12:50 ] Task 1 | epoch 10
[ 12-29 | 12:50 ] Task 1 | epoch 11
[ 12-29 | 12:50 ] Task 1 | epoch 12
[ 12-29 | 12:50 ] Task 1 | epoch 13
[ 12-29 | 12:51 ] Task 1 | epoch 14
[ 12-29 | 12:51 ] Task 1 | epoch 15
[ 12-29 | 12:51 ] Task 1 | epoch 16
[ 12-29 | 12:51 ] Task 1 | epoch 17
[ 12-29 | 12:51 ] Task 1 | epoch 18
[ 12-29 | 12:51 ] Task 1 | epoch 19
[ 12-29

KeyboardInterrupt: 

In [6]:
from datetime import datetime

log_filename = (
        f"results/Task-Poisoning/{datetime.now().strftime('%m-%d-%y-%H-%M-%S')}-{args.dataset}-{args.model}-buf-{args.buffer_size}-severity-{args.poisoning_severity}-cpp-{args.classes_per_poisoning}"
        f"{'-poisoning-type-' + str(args.image_poisoning_type) if args.n_image_poisonings is not None else '-poisoning-percentage-' + str(args.label_flip_percentage) if args.n_label_flip_poisonings is not None else '-no-poisoning'}.json"
    )

AttributeError: 'Namespace' object has no attribute 'poisoning_severity'

In [3]:
import argparse

# Create a Namespace instance
args = argparse.Namespace()

args.seed=None
args.notes=None
args.non_verbose=0
args.disable_log=0
args.validation=0
args.ignore_other_metrics=1
args.debug_mode=0
args.nowand=1
args.wandb_entity='regaz'
args.wandb_project='mammoth'
args.dataset='seq-cifar100'
args.model='der'
args.lr=0.01
args.optim_wd=0.0
args.optim_mom=0.0
args.optim_nesterov=0
args.n_epochs=50
args.batch_size=32
args.distributed='no'
args.poisoning_type=1
args.poisoning_severity=1
args.n_slots=None
args.n_poisonings=1
args.max_classes_per_poisoning=5
args.sequential_poisonings=False
args.alpha=0.5
args.softmax_temp=2
args.buffer_size=5000
args.n_past_poisonings = 1
args.classes_per_poisoning=0


In [5]:
import argparse

# Create a Namespace instance
args = argparse.Namespace()

args.dataset="seq-cifar100-label-poisoning"
args.model="der"
args.n_epochs=50
args.lr=0.1
args.batch_size=32
args.minibatch_size=32
args.buffer_size=5000
args.nowand=1
args.non_verbose=1
args.ignore_other_metrics=1 
args.n_image_poisonings=0
args.seed=42
args.n_label_flip_poisonings=1
args.classes_per_poisoning=0


In [24]:
args = argparse.Namespace()

args.dataset = "seq-cifar100-label-poisoning"
args.model = "er"
args.n_epochs = 50
args.lr = 0.1
args.batch_size = 32
args.minibatch_size = 32
args.buffer_size = 5000
args.nowand = 1
args.non_verbose = 1
args.ignore_other_metrics = 1

In [29]:
import itertools

def argument_combinations(buffer_size, image_poisoning_type, label_flip_percentage):
    """
    Generator that yields every combination of arguments.
    """
    for combination in itertools.product(buffer_size, image_poisoning_type, label_flip_percentage):
        yield {
            "buffer_size": combination[0],
            "image_poisoning_type": combination[1],
            "label_flip_percentage": combination[2],
        }

buffer_size = [200, 500, 5000]
image_poisoning_type = [0, 1, 2, 3, 4]
label_flip_percentage = [25, 50, 75]

In [32]:
i=0
for args in argument_combinations(buffer_size, image_poisoning_type, label_flip_percentage):
    print(args)
    i+=1

{'buffer_size': 200, 'image_poisoning_type': 0, 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 0, 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 0, 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 1, 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 1, 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 1, 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 2, 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 2, 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 2, 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 3, 'label_flip_percentage': 25}
{'buffer_size': 200, 'image_poisoning_type': 3, 'label_flip_percentage': 50}
{'buffer_size': 200, 'image_poisoning_type': 3, 'label_flip_percentage': 75}
{'buffer_size': 200, 'image_poisoning_type': 4, 'label_flip_percentage': 25}

In [34]:
args = argparse.Namespace(
    seed=None,
    notes=None,
    non_verbose=1,
    disable_log=0,
    validation=0,
    ignore_other_metrics=1,
    debug_mode=0,
    nowand=1,
    wandb_entity='regaz',
    wandb_project='mammoth',
    dataset='seq-cifar100-label-poisoning',
    model='er',
    lr=0.1,
    optim_wd=0.0,
    optim_mom=0.0,
    optim_nesterov=0,
    n_epochs=50,
    batch_size=32,
    distributed='no',
    n_image_poisonings=None,
    image_poisoning_type=None,
    n_label_flip_poisonings=None, 
    label_flip_percentage=None,
    poisoning_severity=1, 
    classes_per_poisoning=0, 
    buffer_size=5000, 
    minibatch_size=32, 
    buffer_mode='balanced', 
    #conf_jobnum='80cc9790-1cc7-4ef5-b5b8-7a2574ca06f7', 
    #conf_timestamp='2024-12-28 17:53:09.485930', 
    conf_host='rankine.cis.rit.edu'
)

In [35]:
main(args)

Namespace(seed=None, notes=None, non_verbose=1, disable_log=0, validation=0, ignore_other_metrics=1, debug_mode=0, nowand=1, wandb_entity='regaz', wandb_project='mammoth', dataset='seq-cifar100-label-poisoning', model='er', lr=0.1, optim_wd=0.0, optim_mom=0.0, optim_nesterov=0, n_epochs=2, batch_size=32, distributed='no', n_image_poisonings=None, image_poisoning_type=None, n_label_flip_poisonings=None, label_flip_percentage=None, poisoning_severity=1, classes_per_poisoning=0, buffer_size=5000, minibatch_size=32, buffer_mode='balanced', conf_host='rankine.cis.rit.edu', conf_jobnum='b5e6b21d-706c-402c-b0ea-c82726979e7d', conf_timestamp='2024-12-28 18:04:30.495114')



/home/aa5264/Desktop/CL-Mammoth/venv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "
[ 12-28 | 18:04 ] Task 1 | epoch 0
[ 12-28 | 18:04 ] Task 1 | epoch 1

Metrics for 1 task(s): 	 [Class-IL Acc]: 70.2 % 	 [Task-IL Acc]: 70.2 %

[ 12-28 | 18:04 ] Task 2 | epoch 0
[ 12-28 | 18:04 ] Task 2 | epoch 1

Metrics for 2 task(s): 	 [Class-IL Acc]: 52.1 % 	 [Task-IL Acc]: 64.9 %

[ 12-28 | 18:04 ] Task 3 | epoch 0
[ 12-28 | 18:04 ] Task 3 | epoch 1

Metrics for 3 task(s): 	 [Class-IL Acc]: 34.6 % 	 [Task-IL Acc]: 60.07 %

[ 12-28 | 18:05 ] Task 4 | epoch 0
[ 12-28 | 18:05 ] Task 4 | epoch 1

Metrics for 4 task(s): 	 [Class-IL Acc]: 32.65 % 	 [Task-IL Acc]: 60.85 %

[ 12-28 | 18:05 ] Task 5 | epoch 0
[ 12-28 | 18:05 ] Task 5 | epoch 1

Metrics for 5 task(s): 	 [Class-IL Acc]: 39.76 % 	 [Task-IL Acc]: 

In [5]:

def parse_args():
    parser = ArgumentParser(description='mammoth', allow_abbrev=False)
    parser.add_argument('--model', type=str, required=True,
                        help='Model name.', choices=get_all_models())
    parser.add_argument('--load_best_args', action='store_true',
                        help='Loads the best arguments for each method, '
                             'dataset and memory buffer.')
    torch.set_num_threads(4)
    add_management_args(parser)
    args = parser.parse_known_args()[0]
    mod = importlib.import_module('models.' + args.model)

    if args.load_best_args:
        parser.add_argument('--dataset', type=str, required=True,
                            choices=DATASET_NAMES,
                            help='Which dataset to perform experiments on.')
        if hasattr(mod, 'Buffer'):
            parser.add_argument('--buffer_size', type=int, required=True,
                                help='The size of the memory buffer.')
        args = parser.parse_args()
        if args.model == 'joint':
            best = best_args[args.dataset]['sgd']
        else:
            best = best_args[args.dataset][args.model]
        if hasattr(mod, 'Buffer'):
            best = best[args.buffer_size]
        else:
            best = best[-1]
        get_parser = getattr(mod, 'get_parser')
        parser = get_parser()
        to_parse = sys.argv[1:] + ['--' + k + '=' + str(v) for k, v in best.items()]
        to_parse.remove('--load_best_args')
        args = parser.parse_args(to_parse)
        if args.model == 'joint' and args.dataset == 'mnist-360':
            args.model = 'joint_gcl'
    else:
        get_parser = getattr(mod, 'get_parser')
        parser = get_parser()
        args = parser.parse_args()

    if args.seed is not None:
        set_random_seed(args.seed)

    return args


def main(args=None):
    lecun_fix()
    if args is None:
        args = parse_args()

    os.putenv("MKL_SERVICE_FORCE_INTEL", "1")
    os.putenv("NPY_MKL_FORCE_INTEL", "1")

    # Add uuid, timestamp and hostname for logging
    args.conf_jobnum = str(uuid.uuid4())
    args.conf_timestamp = str(datetime.datetime.now())
    args.conf_host = socket.gethostname()
    dataset = get_dataset(args)

    if args.n_epochs is None and isinstance(dataset, ContinualDataset):
        args.n_epochs = dataset.get_epochs()
    if args.batch_size is None:
        args.batch_size = dataset.get_batch_size()
    if hasattr(importlib.import_module('models.' + args.model), 'Buffer') and args.minibatch_size is None:
        args.minibatch_size = dataset.get_minibatch_size()

    backbone = dataset.get_backbone()
    loss = dataset.get_loss()
    model = get_model(args, backbone, loss, dataset.get_transform())

    if args.distributed == 'dp':
        model.net = make_dp(model.net)
        model.to('cuda:0')
        args.conf_ngpus = torch.cuda.device_count()
    elif args.distributed == 'ddp':
        # DDP breaks the buffer, it has to be synchronized.
        raise NotImplementedError('Distributed Data Parallel not supported yet.')

    if args.debug_mode:
        args.nowand = 1

    # set job name
    setproctitle.setproctitle('{}_{}_{}'.format(args.model, args.buffer_size if 'buffer_size' in args else 0, args.dataset))

    if isinstance(dataset, ContinualDataset):
        train(model, dataset, args)
    else:
        assert not hasattr(model, 'end_task') or model.NAME == 'joint_gcl'
        ctrain(args)


if __name__ == '__main__':
    main()


usage: ipykernel_launcher.py [-h] --model
                             {si,lucir,rpc,ewc_on,xder_ce,fdr,er,derpp,agem,er_ace,mer,gss,agem_r,joint,sgd,lwf_mc,gem,xder_rpc,joint_gcl,pnn,icarl,gdumb,bic,hal,lwf,der,xder}
                             [--load_best_args] [--seed SEED] [--notes NOTES]
                             [--non_verbose {0,1}] [--disable_log {0,1}]
                             [--validation {0,1}]
                             [--ignore_other_metrics {0,1}]
                             [--debug_mode DEBUG_MODE] [--nowand {0,1}]
                             [--wandb_entity WANDB_ENTITY]
                             [--wandb_project WANDB_PROJECT]
ipykernel_launcher.py: error: the following arguments are required: --model


SystemExit: 2

/home/aa5264/Desktop/CL-Mammoth/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
def main(args=None):
    os.putenv("MKL_SERVICE_FORCE_INTEL", "1")
    os.putenv("NPY_MKL_FORCE_INTEL", "1")

    # Add uuid, timestamp and hostname for logging
    args.conf_jobnum = str(uuid.uuid4())
    args.conf_timestamp = str(datetime.datetime.now())
    args.conf_host = socket.gethostname()
    dataset = get_dataset(args)

    if args.n_epochs is None and isinstance(dataset, ContinualDataset):
        args.n_epochs = 2#dataset.get_epochs()
    if args.batch_size is None:
        args.batch_size = dataset.get_batch_size()
    if hasattr(importlib.import_module('models.' + args.model), 'Buffer') and args.minibatch_size is None:
        args.minibatch_size = dataset.get_minibatch_size()
    args.n_epochs=2
    backbone = dataset.get_backbone()
    loss = dataset.get_loss()
    model = get_model(args, backbone, loss, dataset.get_transform())

    if args.distributed == 'dp':
        model.net = make_dp(model.net)
        model.to('cuda:0')
        args.conf_ngpus = torch.cuda.device_count()
    elif args.distributed == 'ddp':
        # DDP breaks the buffer, it has to be synchronized.
        raise NotImplementedError('Distributed Data Parallel not supported yet.')

    if args.debug_mode:
        args.nowand = 1

    # set job name
    setproctitle.setproctitle('{}_{}_{}'.format(args.model, args.buffer_size if 'buffer_size' in args else 0, args.dataset))

    if isinstance(dataset, ContinualDataset):
        train(model, dataset, args)
    else:
        assert not hasattr(model, 'end_task') or model.NAME == 'joint_gcl'
        ctrain(args)




In [8]:





def parse_args():
    parser = ArgumentParser(description='mammoth', allow_abbrev=False)
    parser.add_argument('--model', type=str, required=True,
                        help='Model name.', choices=get_all_models())
    parser.add_argument('--load_best_args', action='store_true',
                        help='Loads the best arguments for each method, '
                             'dataset and memory buffer.')
    torch.set_num_threads(4)
    add_management_args(parser)
    args = parser.parse_known_args()[0]
    mod = importlib.import_module('models.' + args.model)

    if args.load_best_args:
        parser.add_argument('--dataset', type=str, required=True,
                            choices=DATASET_NAMES,
                            help='Which dataset to perform experiments on.')
        if hasattr(mod, 'Buffer'):
            parser.add_argument('--buffer_size', type=int, required=True,
                                help='The size of the memory buffer.')
        args = parser.parse_args()
        if args.model == 'joint':
            best = best_args[args.dataset]['sgd']
        else:
            best = best_args[args.dataset][args.model]
        if hasattr(mod, 'Buffer'):
            best = best[args.buffer_size]
        else:
            best = best[-1]
        get_parser = getattr(mod, 'get_parser')
        parser = get_parser()
        to_parse = sys.argv[1:] + ['--' + k + '=' + str(v) for k, v in best.items()]
        to_parse.remove('--load_best_args')
        args = parser.parse_args(to_parse)
        if args.model == 'joint' and args.dataset == 'mnist-360':
            args.model = 'joint_gcl'
    else:
        get_parser = getattr(mod, 'get_parser')
        parser = get_parser()
        args = parser.parse_args()

    if args.seed is not None:
        set_random_seed(args.seed)

    return args


def main(args=None):
    if args is None:
        args = parse_args()

    os.putenv("MKL_SERVICE_FORCE_INTEL", "1")
    os.putenv("NPY_MKL_FORCE_INTEL", "1")

    # Add uuid, timestamp and hostname for logging
    args.conf_jobnum = str(uuid.uuid4())
    args.conf_timestamp = str(datetime.datetime.now())
    args.conf_host = socket.gethostname()
    dataset = get_dataset(args)

    if args.n_epochs is None and isinstance(dataset, ContinualDataset):
        args.n_epochs = dataset.get_epochs()
    if args.batch_size is None:
        args.batch_size = dataset.get_batch_size()
    if hasattr(importlib.import_module('models.' + args.model), 'Buffer') and args.minibatch_size is None:
        args.minibatch_size = dataset.get_minibatch_size()

    backbone = dataset.get_backbone()
    loss = dataset.get_loss()
    model = get_model(args, backbone, loss, dataset.get_transform())

    if args.distributed == 'dp':
        model.net = make_dp(model.net)
        model.to('cuda:0')
        args.conf_ngpus = torch.cuda.device_count()
    elif args.distributed == 'ddp':
        # DDP breaks the buffer, it has to be synchronized.
        raise NotImplementedError('Distributed Data Parallel not supported yet.')

    if args.debug_mode:
        args.nowand = 1

    # set job name
    setproctitle.setproctitle('{}_{}_{}'.format(args.model, args.buffer_size if 'buffer_size' in args else 0, args.dataset))

    if isinstance(dataset, ContinualDataset):
        train(model, dataset, args)
    else:
        assert not hasattr(model, 'end_task') or model.NAME == 'joint_gcl'
        ctrain(args)


if __name__ == '__main__':
    main()


usage: ipykernel_launcher.py [-h] --model
                             {si,lucir,rpc,ewc_on,xder_ce,fdr,er,derpp,agem,er_ace,mer,gss,agem_r,joint,sgd,lwf_mc,gem,xder_rpc,joint_gcl,pnn,icarl,gdumb,bic,hal,lwf,der,xder}
                             [--load_best_args] [--seed SEED] [--notes NOTES]
                             [--non_verbose {0,1}] [--disable_log {0,1}]
                             [--validation {0,1}]
                             [--ignore_other_metrics {0,1}]
                             [--debug_mode DEBUG_MODE] [--nowand {0,1}]
                             [--wandb_entity WANDB_ENTITY]
                             [--wandb_project WANDB_PROJECT]
ipykernel_launcher.py: error: the following arguments are required: --model


SystemExit: 2

/home/aa5264/Desktop/CL-Mammoth/venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
